In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [1]:
import io, sys, math
import pandas as pd
import numpy as np
from collections import defaultdict

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [2]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [9]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    # a=pd.DataFrame(data,columns=["ab","ac"])
    # y=a["ab"]
    # x=a["ac"].apply(pd.Series)
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [11]:
def softmax(x):
  max_x=max(x) #get the max
  summ=np.sum(np.exp(x-max_x), axis=0) 
  softmax=np.exp(x-max_x)/summ#subtract the max from eqch x-term
  return softmax
    ### FILL CODE

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [12]:
lr=0.01
def sgd(w, data, niter):
    nlabels, dim = w.shape
    loss=0
    for iter in range(niter):
      train_loss=0
      #x=data#.reshape(1,-1)
      for label,x in data:
        h=np.dot(w,x) #hypothesis
        y_pred=softmax(h) #predict using softmax
        train_loss+= np.log(y_pred[label]) #we want to keep track of our losses. We may use it to draw a graph of losses later
        vector=np.zeros(nlabels)#we create a vector of zeros which would be used to assign labels
        vector[label]=1 #assign 1 where the vector is 0
        grad=((y_pred-vector).reshape(nlabels,1)*x.reshape((1,dim))) #gradient of the softmax
        w=w-lr*grad #updating the weights

    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [13]:
def predict(w, x):
  h=np.dot(w,x)
  y_pred=softmax(h)
    ## FILL CODE
  return np.argmax(y_pred) #we take the argmax of the prediction

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [14]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    accuracy=0
    for label, x in valid_data:
      y_pred=predict(w,x) #predict
      if y_pred==label: #update accuracy
        accuracy+=1
    return (accuracy/len(valid_data))*100

In [15]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data= load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data,2)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **


Validation accuracy: 87.800

